<a href="https://colab.research.google.com/github/informatics-isi-edu/facebase-ml-exec/blob/main/notebooks/VGG19_Diagnosis_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the labels to the catalog.  The ROC curve is also calculated and uploaded.

In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "facebase-ml"))

In [2]:
import json
import os
from facebase_ml.facebase_ml import FaceBaseML
import pandas as pd
from pathlib import Path, PurePath
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2024-04-30 23:22:04.595034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 23:22:07.097586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "fb-ml" #@param
host = 'ml.facebase.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-04-30 23:22:09,585 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 23:22:09,586 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.
# Change to your username on the VM

configuration_rid="58-TW70"      # Configuration file for this run.  Needs to be changed for each execution.

In [5]:
FB = FaceBaseML(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
print(f" Initializing model version: {FB.version}")

2024-04-30 23:22:09,655 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 23:22:09,655 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-04-30 23:22:10,152 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post1+git.d2faa8b8.dirty


 Initializing model version: 1.1.0.post1+git.d2faa8b8.dirty


In [6]:
FB.working_dir

PosixPath('/data/maryamahmadii/FaceBaseML_working')

In [7]:
# @title Initiate an Execution
configuration_records = FB.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

2024-04-30 23:22:14,900 - INFO - File [/data/maryamahmadii/FaceBaseML_working/Execution_Metadata/Execution_Config-MusMorph_train_10pct.json] transfer successful. 0.74 KB transferred. Elapsed time: 0:00:00.000058.
2024-04-30 23:22:14,901 - INFO - Verifying MD5 checksum for downloaded file [/data/maryamahmadii/FaceBaseML_working/Execution_Metadata/Execution_Config-MusMorph_train_10pct.json]
2024-04-30 23:22:14,917 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/maryamahmadii/FaceBaseML_working'),
 'vocabs': {'Workflow_Type': [{'name': 'Model Training', 'rid': '58-TC9E'}],
  'Execution_Asset_Type': [{'name': 'Model', 'rid': '58-TC9G'}]},
 'execution_rid': '58-TW98',
 'workflow_rid': '58-TC9M',
 'bag_paths': [PosixPath('/data/58-TC6A_dbe4619ed2511dd8fae878901edefa38419604b264c4c5d3ad32e9d3c5f5944c/Dataset_58-TC6A'),
  PosixPath('/data/58-TCBG_c602a1818329e7ccc455c931f92b767b57e796e6e6ffda510f18f98e6ebf8d9c/Dataset_58-TCBG'),
  PosixPath('/data/58-TCBR_076bc7f6a9e2ee7dad26971e52af4daf36fef312384a327e0368dbfdd1f5ecae/Dataset_58-TCBR')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/maryamahmadii/FaceBaseML_working/Execution_Metadata/Execution_Config-MusMorph_train_10pct.json')}

In [8]:
train_base_dir = configuration_records.bag_paths[0]
valid_base_dir = configuration_records.bag_paths[1]
test_base_dir = configuration_records.bag_paths[2]

biosample_filename = 'data/biosample.csv'
genotype_filename = 'data/genotype.csv'
Train_output_filename = FB.working_dir/'Train_mapped_file.csv'
Val_output_filename = FB.working_dir/'Val_mapped_file.csv'
Test_output_filename = FB.working_dir/'Test_mapped_file.csv'


Train_df, Train_mapped_file = FB.join_and_save_csv(train_base_dir, biosample_filename, genotype_filename, Train_output_filename)
Val_df, Val_mapped_file = FB.join_and_save_csv(valid_base_dir, biosample_filename, genotype_filename, Val_output_filename)
Test_df, Test_mapped_file = FB.join_and_save_csv(test_base_dir, biosample_filename, genotype_filename, Test_output_filename)


In [9]:
# Prepare datasets
dataset_manager = FaceBaseML()

csv_path = Train_mapped_file
images_folder_path = train_base_dir.joinpath('data/assets/Image')
image_paths, labels = dataset_manager.load_images_and_labels(csv_path, images_folder_path)
train_dataset = dataset_manager.prepare_dataset(image_paths, labels, batch_size=50, shuffle=True)

csv_path = Val_mapped_file
images_folder_path = valid_base_dir.joinpath('data/assets/Image')
image_paths, labels = dataset_manager.load_images_and_labels(csv_path, images_folder_path)
validation_dataset = dataset_manager.prepare_dataset(image_paths, labels, batch_size=50, shuffle=True)

csv_path = Test_mapped_file
images_folder_path = test_base_dir.joinpath('data/assets/Image')
image_paths, labels = dataset_manager.load_images_and_labels(csv_path, images_folder_path)
test_dataset = dataset_manager.prepare_dataset(image_paths, labels, batch_size=50, shuffle=True)


2024-04-30 23:22:22,912 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 23:22:22,913 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-04-30 23:22:23,286 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post1+git.d2faa8b8.dirty
2024-04-30 23:22:23.496589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 23:22:24.022942: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so 

In [ ]:
for images, labels in validation_dataset.take(1):
    print("Images shape:", images.shape) 
    print("Labels shape:", labels.shape)
    print("Images dtype:", images.dtype)
    print("Labels dtype:", labels.dtype)


/home/maryamahmadii/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/nibabel/minc2.py:68: UserWarning: Invalid spacing declaration: b'xspace'; assuming regular
  warnings.warn(f'Invalid spacing declaration: {spacing}; assuming regular')
2024-04-30 23:22:54.110793: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 1 of 93
2024-04-30 23:23:09.539911: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 6 of 93
2024-04-30 23:23:32.007568: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 12 of 93
2024-04-30 23:23:53.483703: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 18 of 93
2024-04-30 23:24:12.639718: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shu

In [ ]:
## Training process will start here

Algorithm was trained on cropped images, so take the raw images and bounding boxes and apply, storing the results in the working directory.

Import the actual model code and then run against the input dataset specified in the configuration file.  

In [ ]:
# @title Execute Proecss algorithm (Test model)
from facebase_ml_tools.models.some_file import #some model

with FB.execution(execution_rid=configuration_records.execution_rid) as exec:
  output_path = FB.execution_assets_path/Path("Model_Prediction")


In [ ]:
# @title Plot ROC.


Add the new lables to the catalog using the provided diagnosis tage for this execution.  Also upload any additional assets that were produced by this execution..

In [ ]:
# @title Save Diagnosis


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = FB.execution_upload(configuration_records.execution_rid, False)
